In [455]:
import pandas as pd
import numpy as np


In [466]:
data = pd.read_csv("korean_air_2020_01.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_02.csv")
#data = pd.read_csv("korean_air_2020_03.csv")
#data = pd.read_csv("korean_air_2020_04.csv")
#data = pd.read_csv("korean_air_2020_05.csv")
#data = pd.read_csv("korean_air_2020_6.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_07.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_08.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_09.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_10.csv")

#data = pd.concat([first, second, third, fourth, fifth, sixth])

In [467]:
data = data.dropna()
data.columns = ["a","title","text","repoter","spon", "date"]
date = data.date

In [468]:
#q불용어 지정
#기자이름
repoter = data["repoter"].unique

In [469]:
#신문사 이름
spon = data["spon"].unique

In [470]:
#대표적인 조사
stopwords = ['의','가','이','권','은','들','는','좀','잘','걍','과','도','을','것','를','등','으로','자','에','등','와','한','하다''것','로']

In [471]:
#불용어 병합
stopwords.extend(data.repoter.values)
stopwords.extend(data.spon.values)

In [472]:
len(data)

432

In [473]:
# 한글 외의 모든 데이터 제거 : 기업명이 영어로 되어있는 경우를 어떻게 처리 해야할까?
data = data.text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer

okt = Okt()
X_train = []
X_train2 =[]

for sen in data :
    temp_X = []
    
    # 토큰화
    temp_X = okt.morphs(sen, stem = True) 
    
    # 불용어 제거
    temp_X = [word for word in temp_X if not word in stopwords] 
    
    #모든 토큰들을 리스트에 기사별로 저장
    X_train.append(temp_X)
    
    #모든 토큰들을  하나의 리스트에 저장
    X_train2.extend(temp_X)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
vocab_size = total_cnt - rare_cnt
print('단어 집합의 크기 :',vocab_size)

단어 집합(vocabulary)의 크기 : 8592
등장 빈도가 2번 이하인 희귀 단어의 수: 4133
단어 집합에서 희귀 단어의 비율: 48.10288640595903
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 3.481598379020788
단어 집합의 크기 : 4459


In [474]:
#요일 구하기
import datetime
def getDayName(a):
        D = a.split("/")
        y = int(D[0])
        m = int(D[1])
        d = int(D[2])
        return ['MON','TUE','WED','THU','FRI','SAT','SUN'][datetime.date(y,m,d).weekday()]

In [475]:
weekday =[]
for x in date:
    weekday.append(getDayName(x))
    

In [476]:
date = date.tolist()

In [477]:
DATE = []
for x in date:
    if x not in DATE:
        DATE.append(x)
print(DATE)

['2020/01/31', '2020/01/30', '2020/01/29', '2020/01/28', '2020/01/27', '2020/01/26', '2020/01/25', '2020/01/24', '2020/01/23', '2020/01/22', '2020/01/21', '2020/01/20', '2020/01/19', '2020/01/18', '2020/01/17', '2020/01/16', '2020/01/15', '2020/01/14', '2020/01/13', '2020/01/12', '2020/01/11', '2020/01/10', '2020/01/09', '2020/01/08', '2020/01/07', '2020/01/06', '2020/01/05', '2020/01/01']


In [478]:
#증감 데이터 불러오기
increase_r = pd.read_csv("대비처리.csv")

In [479]:
increase = increase_r[["년/월/일","증감"]]

In [480]:
weekday_frame = pd.DataFrame( {"date" : date,"weekday" : weekday})

In [481]:
 for x in range(0,len(weekday_frame)):
    if weekday_frame.weekday[x] == "SAT":
        monday = DATE.index(weekday_frame.date[x])- 2
        weekday_frame.date[x] = DATE[monday]
        weekday_frame.weekday[x] == "MON"
            
    else :
        if weekday_frame.weekday[x] == "SUN":
            monday = DATE.index(weekday_frame.date[x]) - 1
            weekday_frame.date[x] = DATE[monday]
            weekday_frame.weekday[x] == "MON"
                
        else : pass
                

In [482]:
weekday_frame.date.unique()

array(['2020/01/31', '2020/01/30', '2020/01/29', '2020/01/28',
       '2020/01/27', '2020/01/24', '2020/01/23', '2020/01/22',
       '2020/01/21', '2020/01/20', '2020/01/17', '2020/01/16',
       '2020/01/15', '2020/01/14', '2020/01/13', '2020/01/10',
       '2020/01/09', '2020/01/08', '2020/01/07', '2020/01/06',
       '2020/01/01'], dtype=object)

In [483]:
weekday_frame = weekday_frame.drop_duplicates()

In [484]:
weekday_frame.to_csv("weekday.csv")

In [485]:
#날자를 중심으로 데이터 병합
increase = increase.rename(columns ={ "년/월/일" : "date"})

In [486]:
dic_frame = pd.DataFrame({"data" : X_train, "date" : date})

In [487]:
increase2 = pd.merge(weekday_frame, increase , on = "date")

In [488]:
del increase2["weekday"]

In [489]:
increase2 = increase2.drop_duplicates()

In [490]:
dic_frame2 = pd.merge( dic_frame, increase2, on = "date", how = "left")

In [491]:
dic = dic_frame2.fillna(0)

In [492]:
#딕셔너리 key 생성
key_list = []
for v in X_train2:
    if v not in key_list:
        key_list.append(v)

In [494]:
#예상 소요 시간 300분
#사전 만들기
dictionary = {"<UNK>" : { "횟수" : 0, "점수" : 0, "극성정도" : 0}}

for k in key_list:  #키 리스트안에 있는 단어가 언급이 되는 경우
    count = 0
    score = 0
    for x in range(0,len(dic)):  #한 기사씩 검사한다.
        d = dic.loc[x,:].data
        
        
        #count 는 전체 언급된 횟수, score는 증감의 총합, pority는 총합의 평균
        if k in d:            
            count = count + d.count(k)
            score = score + (dic.loc[x,:]["증감"] * d.count(k))
        
       
        #elif k not in word_list:
        else:
            pass

    polarity  = score/count
    dictionary[k] = { "횟수" : count, "점수" : score, "극성정도" : polarity}  

In [495]:
#기사별 점수 계산

from collections import Counter
scores = []
for x in range (0, len(dic)):
    key = set(dic.data[x])
    score = 0
    count = 0
    for i in key:
        score += (dictionary[i]["극성정도"] * Counter(dic.data[x])[i])
        count += Counter(dic.data[x])[i]
    try :
        score = score/count
        scores.append(score)
        
    except ZeroDivisionError as e:
        print(x,k)
        mis = x

227 미온


In [496]:
#이상치 제거
dic.drop([mis], inplace=True)

In [497]:
dic["score"] = scores

In [499]:
#날자별 점수 평균계산
score_data = dic.groupby(['date'])["score"].mean()

In [504]:
#날자별 평균 데이터 저장
score_data.to_csv("score_data.csv")

In [451]:
daehan_06 = dictionary

In [452]:
import pickle

In [453]:
#딕셔너리 저장
# save data

with open('daehan_06.pickle','wb') as fw:
    pickle.dump(daehan_06, fw)